In [1]:
import os
import pandas as pd
import re

In [2]:
#TODO Make a more dynamic path system later

nts_trip = "/home/trapfishscott/Cambridge24.25/Energy_thesis/Data/NTS_data/UKDA-5340-tab/tab/trip_eul_2002-2023.tab"
nts_i = "/home/trapfishscott/Cambridge24.25/Energy_thesis/Data/NTS_data/UKDA-5340-tab/tab/individual_eul_2002-2023.tab"
nts_vehicle = "/home/trapfishscott/Cambridge24.25/Energy_thesis/Data/NTS_data/UKDA-5340-tab/tab/vehicle_eul_2002-2023.tab"
nts_household = "/home/trapfishscott/Cambridge24.25/Energy_thesis/Data/NTS_data/UKDA-5340-tab/tab/household_eul_2002-2023.tab"
nts_psu = "/home/trapfishscott/Cambridge24.25/Energy_thesis/Data/NTS_data/UKDA-5340-tab/tab/psu_eul_2002-2023.tab"
nts_day = "/home/trapfishscott/Cambridge24.25/Energy_thesis/Data/NTS_data/UKDA-5340-tab/tab/day_eul_2002-2023.tab"

In [3]:
# Load in vehicle df

vehicle_df = pd.read_csv(nts_vehicle, sep="\t",  dtype=str)
i_df = pd.read_csv(nts_i, sep="\t",  dtype=str)


# Load in PSID and Household
household_df = pd.read_csv(nts_household, sep="\t",  dtype=str)
psu_df = pd.read_csv(nts_psu, sep="\t",  dtype=str)

# Load in day
day_df = pd.read_csv(nts_day, sep="\t",  dtype=str)

In [ ]:
# Nuisance columns

for df in [vehicle_df, i_df, household_df, psu_df, day_df]:
    df.drop(columns="SurveyYear", axis=1, inplace=True, errors="ignore")

In [6]:
# Load Trip DF and merging in chunks


chunksize = 100000

merged_chunks = []



for i,trip_df in enumerate(pd.read_csv(nts_trip, sep="\t", chunksize=chunksize, dtype=str)):
    # Filter by car only
    trip_df = trip_df[trip_df["MainMode_B04ID"] == "3"].sample(n=min(len(trip_df), 10000), random_state=42)
    chunk = trip_df.merge(i_df, on="IndividualID")
    chunk = chunk.merge(vehicle_df, on="VehicleID")
    chunk = chunk.merge(psu_df, on="PSUID")
    chunk.drop(columns=["PSUID", "HouseholdID"], axis=1, inplace=True, errors="ignore")
    chunk = chunk.merge(day_df, on="DayID")
    chunk.drop(columns="PSUID", axis=1, inplace=True, errors="ignore")
    chunk = chunk.merge(household_df, on="HouseholdID")
    merged_chunks.append(chunk)

    print(f"\rchunk: {i+1} complete!", end="", flush=True)

merged_df = pd.concat(merged_chunks, ignore_index=True)

chunk: 54 complete!

In [7]:
merged_df.to_pickle("/home/trapfishscott/Cambridge24.25/D200_ML_econ/ProblemSets/Project/data/merged_df.pkl")

In [11]:
merged_df = pd.read_pickle("/home/trapfishscott/Cambridge24.25/D200_ML_econ/ProblemSets/Project/data/merged_df.pkl")

In [13]:
# Taking small df for computational speed
merged_df_duplicate = merged_df.sample(1000)
print(f"Num columns: {len(merged_df_duplicate.columns)}")

Num columns: 465


In [14]:
merged_df_duplicate = merged_df_duplicate.T.drop_duplicates().T
print(f"Num columns: {len(merged_df_duplicate.columns)}")

Num columns: 407


In [22]:
# Finding extra columns

dupe_cols = set(merged_df.columns) - set(merged_df_duplicate.columns)

dupe_cols = list(dupe_cols)


In [59]:
merged_df_analysis = merged_df.drop(columns=dupe_cols, axis=1)

In [60]:
# For speed

merged_df_analysis = merged_df_analysis.sample(50000)

In [61]:
# Dealing with missing values

for col,missing_count in merged_df_analysis.isna().sum().items():
    if missing_count/len(merged_df_analysis) >= 0.001:
        print(f"{col}: {missing_count}")

# Trip start/ end are vital variables so we shall drop them

# TripStartHours: 7042
# TripStartMinutes: 7042
# TripStart: 7042
# TripEndHours: 7396
# TripEndMinutes: 7396
# TripEnd: 7396


TripStartHours: 667
TripStartMinutes: 667
TripStart: 667
TripEndHours: 723
TripEndMinutes: 723
TripEnd: 723
QLeaHous: 39316
FarWalk: 34628
DistWalk: 49976
IntPlane: 11474
DTJbYear: 37695
ReNDNaM_B01ID: 34989
CycMore_B01ID: 34989
Cycle4w_B01ID: 34989
ResMNCy_B01ID: 34989
DVLALength: 8974
VehAnMileage: 70
VehComMile: 2175
VehBusMile: 2175
VehPriMile: 2175
ReliaBus_B01ID: 1960
FrqMetro_B01ID: 1960
RelMetro_B01ID: 1960


In [62]:
vars_to_drop_na = ["TripStartHours", "TripStartMinutes", "TripStart", "TripEndHours", "TripEndMinutes", "TripEnd" ]

merged_df_analysis = merged_df_analysis.dropna(subset=vars_to_drop_na)

In [63]:
for col,missing_count in merged_df_analysis.isna().sum().items():
    if missing_count >= 0.1:
        print(f"{col}: {missing_count}")

QLeaHous: 38693
FarWalk: 34031
DistWalk: 49200
IntPlane: 11389
DTJbYear: 37110
ReNDNaM_B01ID: 34480
CycMore_B01ID: 34480
Cycle4w_B01ID: 34480
ResMNCy_B01ID: 34480
IndividualID_y: 24
DVLALength: 8819
EngineCap: 39
VehAnMileage: 69
VehComMile: 2142
VehBusMile: 2142
VehPriMile: 2142
ReliaBus_B01ID: 1910
FrqMetro_B01ID: 1910
RelMetro_B01ID: 1910


QLeaHous: 413346   -> "How many times did you leave the house yesterday - actual number"
FarWalk: 363596 -> "Time last long walk took - minutes - actual time" # Has a banded version: "Time last long walk took - minutes - banded time"

DistWalk: 523930 -> "Distance of last long walk - miles - actual distance", has a banded version
IntPlane: 120431 -> "Number of international plane trips in last 12 months - actual number", has a banded version
DTJbYear: 395771 -> "Date left last paid job - year element"
SchAgeAcc: 524130 -> "Age first unaccompanied to school - actual age"
ReNDNaM_B01ID: 366718 -> "Main reason do not drive"
ReNDNbM_B01ID: 366718 -> "Main reason do not drive"
CycMore_B01ID: 366718 -> "Amount of cycling compared to this time last year"
Cycle4w_B01ID: 366718 -> "Whether ridden a bicycle during the last 4 weeks"
ResMNCy_B01ID: 366718 -> "Main reason for not cycling more"
DVLALength: 94114 -> "Vehicle length from the DVLA database - mm - actual length", contains a banded version
VehComMile: 22566 -> "Annual vehicle commuting mileage - actual mileage", contains banded
VehBusMile: 22562 -> "Annual vehicle business mileage - actual mileage", contains banded
VehPriMile: 22566 -> "Annual vehicle private mileage - actual mileage", contains banded

---> Drop all

In [64]:
cols_to_drop_missing = [
    "QLeaHous",
    "FarWalk",
    "DistWalk",
    "IntPlane",
    "DTJbYear",
    "ReNDNaM_B01ID",
    "CycMore_B01ID",
    "Cycle4w_B01ID",
    "ResMNCy_B01ID",
    "DVLALength",
    "VehComMile",
    "VehBusMile",
    "VehPriMile"
]

merged_df_analysis = merged_df_analysis.drop(columns=cols_to_drop_missing, axis=1)

In [67]:
for col,missing_count in merged_df_analysis.isna().sum().items():
    if missing_count >= 1:
        print(f"{col}: {missing_count/len(merged_df_analysis)}")



EngineCap: 446
VehAnMileage: 709

dropping all nah


In [66]:
merged_df_analysis = merged_df_analysis.dropna()

In [68]:
merged_df_analysis = merged_df_analysis.copy()

In [71]:
merged_df_analysis.reset_index(drop=True, inplace=True)

In [72]:
merged_df_analysis

,TripID,DayID,IndividualID_x,HouseholdID_x,PSUID_x,PersNo_x,TravDay_x,JourSeq,HowComp_B01ID,SeriesCall_B01ID,...,HHIncQIS2005_B01ID,HHIncQIS2005Eng_B01ID,HHIncQIS2006_B01ID,HHIncQDS2006_B01ID,HHIncQIS2006Eng_B01ID,HHIncQDS2006Eng_B01ID,HHIncQIS2007_B01ID,HHIncQDS2007_B01ID,HHIncQIS2007Eng_B01ID,HHIncQDS2007Eng_B01ID
0,2021032384,2021025751,2021003923,2021001660,2021000213,1,5,6,2,2,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
1,2014088419,2014041097,2014006347,2014002626,2014000260,2,7,3,2,2,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
2,2016166954,2016079072,2016012429,2016005183,2016000531,1,7,1,2,2,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
3,2009206076,2009092476,2009014322,2009006005,2009000445,1,6,4,2,2,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
4,2004164617,2004078912,2004011274,2004004679,2004000364,2,1,2,2,2,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47204,2016135357,2016064824,2016010121,2016004217,2016000431,1,4,3,2,2,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
47205,2013155558,2013082066,2013012679,2013005280,2013000450,1,5,4,2,2,...,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10
47206,2006296663,2006146390,2006020913,2006008757,2006000649,1,6,1,2,2,...,-10,-10,3,3,3,3,-10,-10,-10,-10
47207,2007097569,2007047495,2007006785,2007002845,2007000207,2,7,4,2,2,...,-10,-10,-10,-10,-10,-10,2,2,2,2


In [73]:
# Converting all to float

faulty_cols = []


for col in merged_df_analysis.columns:
    try:
        merged_df_analysis[col] = merged_df_analysis[col].astype(float)

    except:
        print(f"conversion did not work for {col}")
        faulty_cols.append(col)

conversion did not work for TripTravTime
conversion did not work for JTTXSC
conversion did not work for TWSDate
conversion did not work for NumLicHolders


In [81]:
merged_df_analysis.drop(columns="TWSDate", axis=1, inplace=True, errors="ignore")

if "TWSDate" in faulty_cols:
    faulty_cols.remove("TWSDate")

faulty_cols

['TripTravTime', 'JTTXSC', 'NumLicHolders']

In [82]:
faulty_indices = []

for index, row in merged_df_analysis[faulty_cols].iterrows():
    for col in faulty_cols:
        try:
            float(row[col])
        except:
            print(f"{row[col]} is faulty")
            faulty_indices.append(index)



  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty
  is faulty


In [84]:
faulty_indices

[2609,
 5726,
 11608,
 13003,
 20663,
 20663,
 24613,
 31721,
 33024,
 33113,
 33976,
 34283,
 34945,
 36116,
 36116,
 41915,
 42769]

In [85]:
print(f"Length of df with faulty rows: {len(merged_df_analysis.iloc[faulty_indices, :])}")

Length of df with faulty rows: 17


empty " " are causing errors. As there are so few the best solution is to drop

In [86]:
merged_df_analysis = merged_df_analysis.drop(index=faulty_indices)

In [88]:
# Converting all to float


for col in merged_df_analysis.columns:
    merged_df_analysis[col] = merged_df_analysis[col].astype(float)



Succesfully converted everything to float

In [89]:
merged_df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47194 entries, 0 to 47208
Columns: 393 entries, TripID to HHIncQDS2007Eng_B01ID
dtypes: float64(393)
memory usage: 141.9 MB


In [97]:
id_cols = ["TripID", "DayID", "HouseholdID_x", "PSUID_x", "SurveyYear"]

merged_df_analysis.drop(columns=id_cols, axis=1, inplace=True, errors="ignore")

In [98]:
# Exploring correlations
corr_matrix = merged_df_analysis.corr().abs()



In [99]:
# Print header
print(f"{'X':<25} {'Y':<25} {'Value':<10}")
print("="*60)

# Iterate through correlation matrix
for index, row in corr_matrix.iterrows():
    for col in corr_matrix.columns:
        if row[col] > 0.9 and index != col:  # Avoid self-correlations (Value = 1.0)
            print(f"{index:<25} {col:<25} {row[col]:<10.3f}")

# Start by dropping those with perfec correlation

X                         Y                         Value     
IndividualID_x            VehicleID                 1.000     
IndividualID_x            IndividualID_y            1.000     
IndividualID_x            SurveyYear_B01ID          1.000     
IndividualID_x            TravelYear                0.999     
IndividualID_x            TWSYear                   0.999     
IndividualID_x            TWEYear                   0.999     
PersNo_x                  VehMainDriv_B01ID         0.905     
SeriesCall_B01ID          JJXSC                     1.000     
MainMode_B03ID            MainMode_B11ID            0.996     
MainMode_B11ID            MainMode_B03ID            0.996     
TripPurpose_B02ID         TripPurpose_B04ID         0.956     
TripPurpose_B04ID         TripPurpose_B02ID         0.956     
TripStartHours            TripStart                 0.998     
TripStartHours            TripStart_B01ID           1.000     
TripStartHours            TripStart_B02ID           0.9

In [93]:
for col in merged_df_analysis.columns:
    if re.findall("ID", str(merged_df_analysis[col])):
        print(col)

TripID
DayID
IndividualID_x
HouseholdID_x
PSUID_x
HowComp_B01ID
SeriesCall_B01ID
MainMode_B03ID
MainMode_B04ID
MainMode_B11ID
TripPurpFrom_B01ID
TripPurpTo_B01ID
TripPurpose_B01ID
TripPurpose_B02ID
TripPurpose_B04ID
TripStart_B01ID
TripStart_B02ID
TripEnd_B01ID
TripEnd_B02ID
TripDisIncSW_B01ID
TripDisExSW_B01ID
TripTotalTime_B01ID
TripTravTime_B01ID
TripOrigGOR_B02ID
TripDestGOR_B02ID
VehicleID
Interview_B01ID
Age_B01ID
Age_B04ID
Sex_B01ID
OfPenAge_B01ID
HRPRelation_B01ID
MarStat_B01ID
LiveWith_B02ID
EthGroupTS_B02ID
EdAttn1_B01ID
EdAttn2_B01ID
EdAttn3_B01ID
LeaHous_B01ID
NotLea_B01ID
OrdBusFreq_B01ID
CoachFreq_B01ID
TrainFreq_B01ID
TaxiCabFreq_B01ID
BicycleFreq_B01ID
PlaneFreq_B01ID
WalkFreq_B01ID
WhWalk_B01ID
FarWalk_B01ID
DistWalk_B01ID
HTrWalk_B01ID
OwnCycle_B01ID
Cycle12_B01ID
CycRoute_B01ID
IntPlane_B01ID
CarPass_B01ID
DrivLic_B02ID
DrivExp_B01ID
CarAccess_B01ID
CarAccess_B02ID
DrivDisable_B01ID
LastDr_B01ID
PDrivSt_B01ID
ResNDrM_B01ID
PBlueBdg_B01ID
IndIncome2002_B02ID
DTJbMonth

In [48]:
original_vars = ["TripStart", "TripEnd", "TripDisExSW", 
                 "TripDisIncSW" ,"TripTotalTime", "TripTravTime", 
                 "SurveyYear", "NumStages"]

In [49]:
# Drop any vars with suffixes

suffix_cols_to_drop = [
    col for col in merged_df_analysis.columns 
    if any(re.match(f"^{var}.*", col) and col != var for var in original_vars)  # Ensure we don't drop the exact match
]

In [50]:
suffix_cols_to_drop

['NumStages_B01ID',
 'TripStartHours',
 'TripStartMinutes',
 'TripStart_B01ID',
 'TripStart_B02ID',
 'TripEndHours',
 'TripEndMinutes',
 'TripEnd_B01ID',
 'TripEnd_B02ID',
 'TripDisIncSW_B01ID',
 'TripDisExSW_B01ID',
 'TripTotalTime_B01ID',
 'TripTravTime_B01ID',
 'SurveyYear_trip',
 'SurveyYear_i',
 'SurveyYear_x',
 'SurveyYear_y',
 'SurveyYear_B01ID']

In [51]:
merged_df_analysis = merged_df_analysis.drop(columns=suffix_cols_to_drop, axis=1)

In [ ]:
# Exploring correlations
corr_matrix = merged_df_analysis.corr().abs()

# Print header
print(f"{'X':<25} {'Y':<25} {'Value':<10}")
print("="*60)

# Iterate through correlation matrix
for index, row in corr_matrix.iterrows():
    for col in corr_matrix.columns:
        if row[col] > 0.9 and index != col:  # Avoid self-correlations (Value = 1.0)
            print(f"{index:<25} {col:<25} {row[col]:<10.3f}")

# Start by dropping those with perfec correlation

Further columns to drop

* SurveyYear -> TravelYear is off interest
* TripTotalTime --> TripTravTime is of interest
* TripDisIncSW --> TripDisExSW is of interest
* JD --> Another variable for distance, perfectly correlated with TripDisExSW
* JTTXSC, JOTXSC --> Highly correlated with TripTravelTime
* TravelWeekDay_B02ID, TravelWeekDay_B03ID, -->_BO1ID is fully encoded DOW rather than WE WD
* TravelDayTypeOld_B01ID, there is new variable for 2008 onwards

In [56]:
more_cols_to_drop = ["TripDisIncSW", "SurveyYear", "TripTotalTime", "JD", "JTTXSC", "JOTXSC", "TravelWeekDay_B02ID", "TravelWeekDay_B03ID", "TravelDayTypeOld_B01ID"]

In [57]:
# We have Travel

merged_df_analysis = merged_df_analysis.drop(columns=more_cols_to_drop, axis=1)

In [59]:
# Exploring correlations
corr_matrix = merged_df_analysis.corr().abs()

# Print header
print(f"{'X':<25} {'Y':<25} {'Value':<10}")
print("="*60)

# Iterate through correlation matrix
for index, row in corr_matrix.iterrows():
    for col in corr_matrix.columns:
        if row[col] > 0.9 and index != col:  # Avoid self-correlations (Value = 1.0)
            print(f"{index:<25} {col:<25} {row[col]:<10.3f}")

# Start by dropping those with perfec correlation

X                         Y                         Value     
SeriesCall_B01ID          JJXSC                     1.000     
MainMode_B03ID            MainMode_B11ID            0.996     
MainMode_B11ID            MainMode_B03ID            0.996     
TripPurpose_B02ID         TripPurpose_B04ID         0.955     
TripPurpose_B04ID         TripPurpose_B02ID         0.955     
TripStart                 TripEnd                   0.981     
TripEnd                   TripStart                 0.981     
TripOrigGOR_B02ID         TripDestGOR_B02ID         0.979     
TripDestGOR_B02ID         TripOrigGOR_B02ID         0.979     
JJXSC                     SeriesCall_B01ID          1.000     
Interview_B01ID           BusDiffSum_B01ID          0.907     
Interview_B01ID           InterviewN_B01ID          0.988     
Interview_B01ID           EScooter_B01ID            0.993     
Interview_B01ID           PSUCountry_B01ID          0.905     
Age_B01ID                 Age_B04ID                 0.9

In [105]:
# encoding num trips
num_trips_mapping = dict(zip(merged_df_analysis.groupby("IndividualID_x")["JourSeq"].max().index, merged_df_analysis.groupby("IndividualID_x")["JourSeq"].max().values))
merged_df_analysis["NumTrips"] = merged_df_analysis["IndividualID_x"].map(num_trips_mapping)

/tmp/ipykernel_9649/1483458916.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df_analysis["NumTrips"] = merged_df_analysis["IndividualID_x"].map(num_trips_mapping)


In [112]:
# Making a trip purpouse mapping

trip_purpose_mapping = {
    1: "Work", 2: "Other", 3: "Other", 4: "Other", 5: "Other",
    6: "Other", 7: "Other", 8: "Other", 9: "Other", 10: "Other",
    11: "Other", 12: "Other", 13: "Other", 14: "Other", 15: "Other",
    16: "Other", 17: "Other", 18: "Other", 19: "Other", 20: "Other",
    21: "Other", 22: "Other", 23: "Home",
}

In [ ]:
# Removing Values for  Dead or Na

merged_df_analysis = merged_df_analysis[~merged_df_analysis["TripPurpFrom_B01ID"].isin([-8,-10])]
merged_df_analysis = merged_df_analysis[~merged_df_analysis["TripPurpTo_B01ID"].isin([-8,-10])]

In [113]:
merged_df_analysis["TripPurpFrom_B01ID"] = merged_df_analysis["TripPurpFrom_B01ID"].map(trip_purpose_mapping)
merged_df_analysis["TripPurpTo_B01ID"] = merged_df_analysis["TripPurpTo_B01ID"].map(trip_purpose_mapping)

In [115]:
merged_df_analysis["TripType"] = list(zip(merged_df_analysis["TripPurpFrom_B01ID"], merged_df_analysis["TripPurpTo_B01ID"]))

In [120]:
trip_type_mapping = {}

for i,type in enumerate(merged_df_analysis["TripType"].unique()):
    trip_type_mapping[type] = i
    
print(f"Trip type mapping")
for k,v in trip_type_mapping.items():
    print(f"{k}: {v}")

Trip type mapping
('Other', 'Home'): 0
('Home', 'Other'): 1
('Work', 'Home'): 2
('Work', 'Other'): 3
('Home', 'Work'): 4
('Other', 'Other'): 5
('Other', 'Work'): 6


In [121]:
merged_df_analysis["TripType"] = merged_df_analysis["TripType"].map(trip_type_mapping)

In [131]:
for k,v in merged_df_analysis.dtypes.items():
    if v == "object":
        print(k, v)

merged_df_analysis.drop(columns=["TripPurpFrom_B01ID", "TripPurpTo_B01ID"], axis=1, inplace=True, errors="ignore")


TripPurpFrom_B01ID object
TripPurpTo_B01ID object


In [ ]:
vars_of_interest = ["TripStart", "TripEnd", "TripDisExSW", ]

In [ ]:
merged_df_analysis["TripDisExSW"]

In [62]:
# Predicting trip distance

corr_matrix["TripDisExSW"].sort_values(ascending=False)

TripDisExSW            1.000000
TripTravTime           0.897463
VehAnMileage           0.215814
VehAnMileage_B01ID     0.191324
IndIncome2002_B02ID    0.119539
                         ...   
MainMode_B04ID              NaN
DrivLic_B02ID               NaN
SchDly_B01ID                NaN
SchTrav_B01ID               NaN
SchAcc_B01ID                NaN
Name: TripDisExSW, Length: 239, dtype: float64

In [ ]:
## Saving to pickle next step is to try and predict Distance

merged_df_analysis.to_pickle("/home/trapfishscott/Cambridge24.25/D200_ML_econ/ProblemSets/Project/data/merged_df_analysis.pkl")

: 